In [1]:
import pyModeS as pms
import pandas as pd
import base64
from preprocess.decoder import Decoder

Lectura del fichero de despegues

In [2]:
datos  = pd.read_csv('test.csv', sep=';')
datos = datos.drop(columns=['Unnamed: 2'])
datos

,ts_kafka,message
0,1734667201534,oAANuYAVzS4ABLg3aB0=
1,1734667201534,kDQjSSgABjG4hW4fTbY=
2,1734667201534,XTxUM/+DTw==
3,1734667201534,kDQjShBRgxXDaCAWeDM=
4,1734667201534,jTRV1fgjAAIASbigrIs=
...,...,...
8354,1734667796655,IAAAALRFCw==
8355,1734667796655,IAAAALRFCw==
8356,1734667796655,jTlkQ5kItjeQCBj8Xms=
8357,1734667796655,IAAGM27mPA==


Los datos correspenden a la lectura que ha realizado la antena durante 1 minuto.

Son poco más de 8k filas, y la informacion de cada despeque se encuentra en la columna message, formateada en base 64.

Almacenaremos los datos en diccionarios.

In [3]:
hola = pms.tell(Decoder.base64toHex(datos.loc[0, "message"]))

                     Message: A0000DB98015CD2E0004B837681D 
                ICAO address: AA5F41 
             Downlink Format: 20 
                    Protocol: Mode-S Comm-B altitude reply 
                    Altitude: 21225 feet
                         BDS: BDS50 (Track and turn report) 
                  Roll angle: 0.0 degrees
                 Track angle: 130.4296875 degrees
                  Track rate: None degree/second
                Ground speed: 368 knots
               True airspeed: 368 knots


In [4]:
Decoder.processMessage(datos.loc[0, "message"], datos.loc[0,"ts_kafka"])

{'Timestamp (kafka)': 1734667201534,
 'Timestamp (date)': datetime.datetime(2024, 12, 20, 5, 0, 1, 534000),
 'Message (base64)': 'oAANuYAVzS4ABLg3aB0=',
 'Message (hex)': 'A0000DB98015CD2E0004B837681D',
 'ICAO': 'AA5F41',
 'Downlink Format': 20,
 'BDS': 'BDS50',
 'Roll angle (deg)': 0.0,
 'True track angle (deg)': 130.4296875,
 'Ground speed (kt)': 368,
 'Track angle rate (deg/sec)': None,
 'True airspeed (kt)': 368,
 'Altitude (ft)': 21225}

ModeS supports the decoding of following types of messages:

DF4 / DF20: Altitude code

DF5 / DF21: Identity code (squawk code)

DF17 / DF18: Automatic Dependent Surveillance-Broadcast (ADS-B)

TC=1-4 / BDS 0,8: Aircraft identification and category

TC=5-8 / BDS 0,6: Surface position

TC=9-18 / BDS 0,5: Airborne position

TC=19 / BDS 0,9: Airborne velocity

TC=28 / BDS 6,1: Airborne status [to be implemented]

TC=29 / BDS 6,2: Target state and status information [to be implemented]

TC=31 / BDS 6,5: Aircraft operational status [to be implemented]

DF20 / DF21: Mode-S Comm-B messages

BDS 1,0: Data link capability report

BDS 1,7: Common usage GICB capability report

BDS 2,0: Aircraft identification

BDS 3,0: ACAS active resolution advisory

BDS 4,0: Selected vertical intention

BDS 4,4: Meteorological routine air report (experimental)

BDS 4,5: Meteorological hazard report (experimental)

BDS 5,0: Track and turn report

BDS 6,0: Heading and speed report

In [8]:
Decoder.processMessage(datos.loc[1, "message"], datos.loc[1,"ts_kafka"])

{'Timestamp (kafka)': 1734667201534,
 'Timestamp (date)': datetime.datetime(2024, 12, 20, 5, 0, 1, 534000),
 'Message (base64)': 'kDQjSSgABjG4hW4fTbY=',
 'Message (hex)': '9034234928000631B8856E1F4DB6',
 'ICAO': '342349',
 'Downlink Format': 18,
 'Typecode': 5,
 'Surface velocity': (None, None, 0, 'GS'),
 'Position with ref (RADAR)': (40.49779342392743, -3.5578536987304688),
 'Airborne position with ref (RADAR)': (39.957275390625, -6.049596613103693),
 'Surface position with ref (RADAR)': (40.49779342392743, -3.5578536987304688),
 'Altitude (ft)': 0}

In [10]:
Decoder.processMessage(datos.loc[3, "message"], datos.loc[3,"ts_kafka"])

{'Timestamp (kafka)': 1734667201534,
 'Timestamp (date)': datetime.datetime(2024, 12, 20, 5, 0, 1, 534000),
 'Message (base64)': 'kDQjShBRgxXDaCAWeDM=',
 'Message (hex)': '9034234A10518315C36820167833',
 'ICAO': '34234A',
 'Downlink Format': 18,
 'Typecode': 2,
 'Callsign': 'TXLU06__'}

In [7]:
df = datos.copy()
df_resultado = df.apply(lambda x: Decoder.processMessage(x["message"], x["ts_kafka"]), axis=1).apply(pd.Series)
df_resultado

,Timestamp (kafka),Timestamp (date),Message (base64),Message (hex),ICAO,Downlink Format,BDS,Roll angle (deg),True track angle (deg),Ground speed (kt),...,Wind shear level (0-3),Microburst level (0-3),Icing level (0-3),Wake vortex level (0-3),Static air temperature (C),Average static pressure (hPa),Radio height (ft),Overlay capability,Wind speed (kt) and direction (true) (deg),Humidity (%)
0,1734667201534,2024-12-20 05:00:01.534,oAANuYAVzS4ABLg3aB0=,A0000DB98015CD2E0004B837681D,AA5F41,20,BDS50,0.0,130.429688,368.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1734667201534,2024-12-20 05:00:01.534,kDQjSSgABjG4hW4fTbY=,9034234928000631B8856E1F4DB6,342349,18,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1734667201534,2024-12-20 05:00:01.534,XTxUM/+DTw==,5D3C5433FF834F,3C5433,11,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1734667201534,2024-12-20 05:00:01.534,kDQjShBRgxXDaCAWeDM=,9034234A10518315C36820167833,34234A,18,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1734667201534,2024-12-20 05:00:01.534,jTRV1fgjAAIASbigrIs=,8D3455D5F82300020049B8A0AC8B,3455D5,17,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8354,1734667796655,2024-12-20 05:09:56.655,IAAAALRFCw==,20000000B4450B,342354,4,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8355,1734667796655,2024-12-20 05:09:56.655,IAAAALRFCw==,20000000B4450B,342354,4,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8356,1734667796655,2024-12-20 05:09:56.655,jTlkQ5kItjeQCBj8Xms=,8D3964439908B637900818FC5E6B,396443,17,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8357,1734667796655,2024-12-20 05:09:56.655,IAAGM27mPA==,200006336EE63C,346419,4,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df_resultado.columns

Index(['Timestamp (kafka)', 'Timestamp (date)', 'Message (base64)',
       'Message (hex)', 'ICAO', 'Downlink Format', 'BDS', 'Roll angle (deg)',
       'True track angle (deg)', 'Ground speed (kt)',
       'Track angle rate (deg/sec)', 'True airspeed (kt)', 'Altitude (ft)',
       'Typecode', 'Surface velocity', 'Position with ref (RADAR)',
       'Airborne position with ref (RADAR)',
       'Surface position with ref (RADAR)', 'Callsign',
       'MCP/FCU selected altitude (ft)', 'FMS selected altitude (ft)',
       'Barometric pressure (mb)', 'Velocity', 'Speed heading',
       'Airborne velocity', 'Magnetic heading (deg)',
       'Indicated airspeed (kt)', 'Mach number (-)',
       'Barometric altitude rate (ft/min)', 'Inertial vertical speed (ft/min)',
       'Squawk code', 'GICB capability', 'Turbulence level (0-3)',
       'Wind shear level (0-3)', 'Microburst level (0-3)', 'Icing level (0-3)',
       'Wake vortex level (0-3)', 'Static air temperature (C)',
       'Average static

*IDENTIFICACION Y CATEGORIA DEL AEREONAVE*

In [17]:
df_resultado[df_resultado['Typecode']==4]

,Timestamp (kafka),Timestamp (date),Message (base64),Message (hex),ICAO,Downlink Format,BDS,Roll angle (deg),True track angle (deg),Ground speed (kt),...,Wind shear level (0-3),Microburst level (0-3),Icing level (0-3),Wake vortex level (0-3),Static air temperature (C),Average static pressure (hPa),Radio height (ft),Overlay capability,Wind speed (kt) and direction (true) (deg),Humidity (%)
59,1734667201736,2024-12-20 05:00:01.736,jeBliCUFIfHHPSCuhHw=,8DE06588250521F1C73D20AE847C,E06588,17,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
169,1734667212523,2024-12-20 05:00:12.523,japfQSUEEzfTiCCIrgg=,8DAA5F4125041337D3882088AE08,AA5F41,17,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
268,1734667219077,2024-12-20 05:00:19.077,jTxUMyUINPTEWCBjtas=,8D3C5433250834F4C4582063B5AB,3C5433,17,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
358,1734667231175,2024-12-20 05:00:31.175,jeBliCUFIfHHPSCuhHw=,8DE06588250521F1C73D20AE847C,E06588,17,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
506,1734667235511,2024-12-20 05:00:35.511,jUB2LCNNdTHjCCBs0l0=,8D40762C234D7531E308206CD25D,40762C,17,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7972,1734667770568,2024-12-20 05:09:30.568,jOBliCUFIfHHPSD29QQ=,8CE06588250521F1C73D20F6F504,E06588,17,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7993,1734667772481,2024-12-20 05:09:32.481,jTR1TSUkIXDHHiBV2ek=,8D34754D25242170C71E2055D9E9,34754D,17,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8062,1734667784253,2024-12-20 05:09:44.253,jTRkGSUkIXDHDKBbFjY=,8D34641925242170C70CA05B1636,346419,17,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8146,1734667787582,2024-12-20 05:09:47.582,jTR1TSUkIXDHHiBV2ek=,8D34754D25242170C71E2055D9E9,34754D,17,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
msg = "8D40762C234D7531E308206CD25D	"  # Mensaje ADS-B de prueba

wtc = Decoder.getWakeTurbulenceCategory(msg)
print("Categoría de Turbulencia de Estela:", wtc)


Categoría de Turbulencia de Estela: M (Medium)
